# Serviço de inferência

In [2]:
!pip install ultralytics azure-storage-blob azure-cosmos huggingface-hub pillow

In [ ]:
import os
import time
import glob
import shutil
import uuid
import json
import cv2
from PIL import Image
from io import BytesIO
from zipfile import ZipFile
from datetime import datetime

from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from PIL import Image
from azure.storage.blob import BlobServiceClient
from azure.cosmos import CosmosClient


# ─── CONFIGURAÇÃO ──────────────────────────────────────────────────────────────
AZ_CONN_STR         = "<connection-string>"          # Connection string do Storage
INPUT_CONTAINER     = "input-container"                  # Container de onde vêm os ZIPs
OUTPUT_CONTAINER    = "predict-source"                   # Container onde saem as imagens anotadas
PROCESSED_PREFIX    = "ProcessadoComSucesso/"            # Pasta virtual p/ zips já processados
LOCAL_TMP           = "/content/tmp"                     # Pasta temporária local
INTERVALO           = 60                                  # Checa novo ZIP a cada 60 segundos

# Cosmos DB (opcional, retire se não usar)
COSMOS_ENDPOINT     = "https://<cosmos-name-server>.documents.azure.com:443/"
COSMOS_KEY          = "<cosmos-key>"
COSMOS_DATABASE     = "GreeningDetection"
COSMOS_CONTAINER    = "Predicoes"

# URL base do seu Blob Storage (p/ montar a URL pública)
STORAGE_ACCOUNT_URL = "https://<blob-name-service>.blob.core.windows.net"

# ─── PREPARA AMBIENTE LOCAL ────────────────────────────────────────────────────
os.makedirs(LOCAL_TMP, exist_ok=True)

# ─── CONEXÕES AZURE ─────────────────────────────────────────────────────────────
blob_service    = BlobServiceClient.from_connection_string(AZ_CONN_STR)
input_client    = blob_service.get_container_client(INPUT_CONTAINER)
output_client   = blob_service.get_container_client(OUTPUT_CONTAINER)

cosmos_client   = CosmosClient(COSMOS_ENDPOINT, COSMOS_KEY)
db_client       = cosmos_client.get_database_client(COSMOS_DATABASE)
cosmos_container = db_client.get_container_client(COSMOS_CONTAINER)


In [6]:
# ─── CARREGA MODELO UMA ÚNICA VEZ ──────────────────────────────────────────────
model_path = hf_hub_download(
    repo_id="cuidacitrus/yolov8n-greening-detector",
    filename="v3_best.pt"
)
model = YOLO(model_path)

In [23]:
# ─── FUNÇÃO PRINCIPAL DE PROCESSAMENTO ─────────────────────────────────────────
def processar_zip(blob_name: str):
    print(f"\n[+] Iniciando processamento de: {blob_name}")

    # 1) Download do ZIP
    local_zip = os.path.join(LOCAL_TMP, os.path.basename(blob_name))
    with open(local_zip, "wb") as f:
        f.write(input_client.download_blob(blob_name).readall())

    # 2) Extrai para pasta temporária
    extrair_dir = os.path.join(LOCAL_TMP, "extracted")
    if os.path.exists(extrair_dir):
        shutil.rmtree(extrair_dir)
    os.makedirs(extrair_dir, exist_ok=True)

    with ZipFile(local_zip, "r") as z:
        z.extractall(extrair_dir)

    # 3) Carrega metadata.json (se existir)
    metadata_map = {}
    meta_file = os.path.join(extrair_dir, "metadata.json")
    if os.path.exists(meta_file):
        with open(meta_file, "r") as f:
            data = json.load(f)

        raw = data.get("locations")
        if isinstance(raw, list):
            # achata se for lista de listas
            if raw and isinstance(raw[0], list):
                items = [obj for sub in raw for obj in sub]
            else:
                items = raw
        else:
            items = []

        for item in items:
            if isinstance(item, dict) and "filename" in item:
                metadata_map[item["filename"]] = item

    # 4) Executa inferência e processa cada imagem
    for img_path in glob.glob(os.path.join(extrair_dir, "*")):
        if not img_path.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        # 4.1) Inferência
        res = model.predict(source=img_path, conf=0.75, verbose=False)[0]

        # 4.2) Gera a imagem anotada em memória
        annotated_bgr = res.plot()
        annotated_rgb = cv2.cvtColor(annotated_bgr, cv2.COLOR_BGR2RGB)

        img_pil = Image.fromarray(annotated_rgb)
        buf     = BytesIO()
        img_pil.save(buf, format="JPEG")
        buf.seek(0)

        # 4.3) Upload da imagem anotada
        fname     = os.path.basename(img_path)
        blob_id   = f"{fname}"
        output_client.upload_blob(name=blob_id, data=buf, overwrite=True)
        url_image = f"{STORAGE_ACCOUNT_URL}/{OUTPUT_CONTAINER}/{blob_id}"

        # 4.4) Extrai detecções “Greening” e grava no Cosmos (se houver)
        detections = []
        for box in res.boxes:
            label = res.names[int(box.cls)]
            if label.lower() == "greening":
                detections.append({
                    "label":      label,
                    "confidence": float(box.conf),
                    "bbox":       [*map(float, box.xyxy[0].tolist())]
                })

        if detections:
            meta = metadata_map.get(fname, {})
            item = {
                "id":           fname,
                "detections":   detections,
                "latitude":     meta.get("latitude"),
                "longitude":    meta.get("longitude"),
                "url_image":    url_image,
                "processed_at": datetime.utcnow().isoformat()
            }
            cosmos_container.upsert_item(item)

    # 5) Move ZIP original para ProcessadoComSucesso e apaga o original
    new_blob = PROCESSED_PREFIX + os.path.basename(blob_name)

    # cria BlobClient para origem e destino
    source_blob = input_client.get_blob_client(blob_name)
    dest_blob   = input_client.get_blob_client(new_blob)

    # inicia a cópia
    copy_props = dest_blob.start_copy_from_url(source_blob.url)
    print(f"  → Iniciado copy: status={copy_props['copy_status']}")

    # espera opcionalmente até a cópia completar
    props = dest_blob.get_blob_properties()
    while props.copy.status == "pending":
        time.sleep(1)
        props = dest_blob.get_blob_properties()

    # deleta o blob original
    source_blob.delete_blob()
    print(f"  → Blob original '{blob_name}' movido para '{new_blob}' e deletado.")

    # 6) Limpeza local
    os.remove(local_zip)
    shutil.rmtree(extrair_dir)
    print(f"[+] Concluído o processamento de: {blob_name}")

In [25]:
# ─── LOOP INFINITO ──────────────────────────────────────────────────────────────
print(f"▶ Serviço rodando: checando por novos ZIPs a cada {INTERVALO}s ...")
try:
    while True:
        for b in input_client.list_blobs():
            name = b.name
            if not name.lower().endswith(".zip"):
                continue
            if name.startswith(PROCESSED_PREFIX):
                continue
            processar_zip(name)
        time.sleep(INTERVALO)

except KeyboardInterrupt:
    print("⏹ Serviço interrompido pelo usuário.")

▶ Serviço rodando: checando por novos ZIPs a cada 60s ...
⏹ Serviço interrompido pelo usuário.
